###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

In [3]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
URL = "https://www.labirratorium.com/es/67-cervezas-por-estilo?page="

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [ ]:
# La web tiene 33 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [8]:
resp = requests.get('https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html')
soup = bs(resp.text, 'lxml')

In [40]:
# Extraemos la tabla localizada como etiqueta dl con class=data-sheet
table = soup.find('dl', attrs={'class':'data-sheet'})

# Dentro de la tabla, la cabecera son etiq 'dt' y el contenido 'dd'
dt_tags = table.find_all('dt')
dd_tags = table.find_all('dd')

ls_dt = [tag.string for tag in dt_tags]
ls_dd = [tag.string for tag in dd_tags]

df_ind = pd.DataFrame(data=ls_dd, index=ls_dt)
df_ind = df_ind.T

desc_tag = soup.find('div', attrs={'class':'product-description'})
df_ind['Desc'] = desc_tag.string

df_ind

,Estilo,Origen,% Alc.,% Alc. Exacto,Otros ingredientes,Volumen (cl),Tipo Fermentación,Maltas,IBU,Color,Envase,OLD-REF,Desc
0,KRIEK,Bélgica,ALTO (6-9%),6.5,Cerezas Naturales,37.5 Cl,Lambic (Fermentación espontánea o salvaje),Cebada y Trigo,0-25 Amargor bajo,Rojiza,Botella,5412783053220,Cerveza de fermentación espontánea (Lambic) de...


In [49]:
def get_beer_info(url):
    resp = requests.get(url)
    soup = bs(resp.text, 'lxml')

    # Extraemos la tabla localizada como etiqueta dl con class=data-sheet
    table = soup.find('dl', attrs={'class':'data-sheet'})

    # Dentro de la tabla, la cabecera son etiq 'dt' y el contenido 'dd'
    dt_tags = table.find_all('dt')
    dd_tags = table.find_all('dd')

    # Extraemos el texto/contenido de dentro de las etiquetas
    ls_dt = [tag.string for tag in dt_tags]
    ls_dd = [tag.string for tag in dd_tags]

    # Construimos el dataframe. Generamos como columna y trasponemos
    df_ind = pd.DataFrame(data=ls_dd, index=ls_dt)
    df_ind = df_ind.T

    # Añadimos la descripcion que está en otra parte de la web
    desc_tag = soup.find('div', attrs={'class':'product-description'})
    df_ind['Desc'] = desc_tag.string

    name_tag = soup.find('h1', attrs={'class':'h1 product-detail-name'})
    df_ind['Nombre'] = name_tag.string

    price_tag = soup.find('div', attrs={'class':'current-price'}).span['content']
    df_ind['Precio'] = price_tag

    img_link = soup.find('img', attrs={'class':'js-qv-product-cover'})['src']
    df_ind['img_url'] = img_link
    return df_ind

In [47]:
get_beer_info('https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html')

,Estilo,Origen,% Alc.,% Alc. Exacto,Otros ingredientes,Volumen (cl),Tipo Fermentación,Maltas,IBU,Color,Envase,OLD-REF,Desc,Nombre,Precio
0,KRIEK,Bélgica,ALTO (6-9%),6.5,Cerezas Naturales,37.5 Cl,Lambic (Fermentación espontánea o salvaje),Cebada y Trigo,0-25 Amargor bajo,Rojiza,Botella,5412783053220,Cerveza de fermentación espontánea (Lambic) de...,"Boon Oude Kriek 37,5cl",7.15


In [50]:
get_beer_info('https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html')

,Estilo,Origen,% Alc.,% Alc. Exacto,Volumen (cl),Tipo Fermentación,Maltas,IBU,IBU Exacto,Color,Envase,OLD-REF,Desc,Nombre,Precio,img_url
0,MUNICH HELLES,Alemania,MEDIO (4-6%),5.2,50 Cl,Lager (Baja Fermentación),Cebada,0-25 Amargor bajo,12,Amarillo pálido,Botella,4105250022003,None,Augustiner Lagerbier Hell,2.75,https://www.labirratorium.com/367-large_defaul...


Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame con 

Nombre,	precio,	descripcion corta, descripción larga, ruta imagen, marca, barcode,	Estilo, Origen, % Alc., % Alc. Exacto, Volumen (Cl), Tipo Fermentación, IBU, IBU Exacto, Color, Envase

In [51]:
# Hacemos sopa de la pagina principal
resp = requests.get(URL)
soup = bs(resp.text, 'lxml')

In [53]:
h3_tags = soup.find_all('h3', attrs={'class':'product-title'})
ls_hrefs = [tag.a['href'] for tag in h3_tags]
ls_hrefs

['https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html',
 'https://www.labirratorium.com/es/cerveza-lambic/287-oud-beersel-kriek75.html',
 'https://www.labirratorium.com/es/inicio/182-weihenstephaner-korbinian.html',
 'https://www.labirratorium.com/es/cerveza-lager/180-schlenkerla-rauchbier-marzen.html']

In [54]:
ls_df = []
for link in ls_hrefs:
    df_i = get_beer_info(link)
    ls_df.append(df_i)

df = pd.concat(ls_df, ignore_index=True)


,Estilo,Origen,% Alc.,% Alc. Exacto,Otros ingredientes,Volumen (cl),Tipo Fermentación,Maltas,IBU,Color,Envase,OLD-REF,Desc,Nombre,Precio,img_url,IBU Exacto,Lúpulos
0,KRIEK,Bélgica,ALTO (6-9%),6.5,Cerezas Naturales,37.5 Cl,Lambic (Fermentación espontánea o salvaje),Cebada y Trigo,0-25 Amargor bajo,Rojiza,Botella,5412783053220,Cerveza de fermentación espontánea (Lambic) de...,"Boon Oude Kriek 37,5cl",7.15,https://www.labirratorium.com/19351-large_defa...,NaN,NaN
1,MUNICH HELLES,Alemania,MEDIO (4-6%),5.2,NaN,50 Cl,Lager (Baja Fermentación),Cebada,0-25 Amargor bajo,Amarillo pálido,Botella,4105250022003,None,Augustiner Lagerbier Hell,2.75,https://www.labirratorium.com/367-large_defaul...,12,NaN
2,NaN,Alemania,NaN,12,NaN,33,Lager (Baja Fermentación),NaN,0-25 Amargor bajo,NaN,NaN,4003669018269,"Cerveza de color medio oscuro, aromas alcohóli...",Schneider Aventinus Weizen-Eisbock,2.85,https://www.labirratorium.com/366-large_defaul...,NaN,NaN
3,RAUCHBIER,Alemania,MEDIO (4-6%),5.2,NaN,50 Cl,Ale (Alta Fermentación),Trigo y Cebada,0-25 Amargor bajo,Marrón,Botella,4037458000159,Rauchbier (cerveza ahumada) de trigo con 5.2% ...,Aecht Schlenkerla Rauchbier Weizen,2.7,https://www.labirratorium.com/364-large_defaul...,20,NaN
4,AMERICAN LAGER,EEUU,MEDIO (4-6%),4.8,NaN,33 Cl,Samuel Adams lager,Samuel Adams two-row pale malt blend y Caramel 60,25-50 Amargor medio,Ámbar,Botella,5012686070005,None,Samuel Adams Boston Lager,2.45,https://www.labirratorium.com/343-large_defaul...,30,Hallertau Mittelfrueh y Tettnang Tettnanger No...
5,DUBBEL,Bélgica,ALTO (6-9%),7,NaN,33 Cl,Ale (Alta Fermentación),Cebada,None,Marrón,Botella,5412343152332,Cerveza Trapista belga con 7% ABV de color mar...,Westmalle Dubbel,2.5,https://www.labirratorium.com/102-large_defaul...,24,NaN
6,None,Bélgica,ALTO (6-9%),6.2,NaN,25 Cl,Ale (Alta Fermentación),Cebada,0-25 Amargor bajo,Ámbar,Botella,5411364151119,None,Duchesse De Bourgogne,2.95,https://www.labirratorium.com/12300-large_defa...,NaN,NaN
7,NaN,Bélgica,ALTO (6-9%),8.4,NaN,33 Cl,Ale (Alta Fermentación),"Cebada, Avena y Trigo",0-25 Amargor bajo,Amarillo pálido,Botella,54050082,None,Tripel Karmeliet 33 cl.,2.7,https://www.labirratorium.com/81-large_default...,16,NaN
8,TRIGO ALEMÁN (WEIZENDOPPELBOCK),Alemania,ALTO (6-9%),"7,7",NaN,50 Cl,Ale (Alta Fermentación),Trigo y Cebada,0-25 Amargor bajo,Amarillo dorado,Botella,4105120004108,None,Weihenstephaner Vitus,2.95,https://www.labirratorium.com/43-large_default...,NaN,NaN
9,KRIEK,Belgica,MEDIO (4-6%),6,Cerezas,75,Lambic (Fermentación espontánea o salvaje),Cebada y Trigo,0-25 Amargor bajo,Rojiza,Botella,5425018070138,None,Oud Beersel Oude Kriek Vieille 75 cl.,15.25,https://www.labirratorium.com/620-large_defaul...,NaN,NaN


In [59]:
df = df[['Nombre', 'Precio',	'img_url', 'Estilo', 'Origen', '% Alc.', 
    '% Alc. Exacto', 'Otros ingredientes', 'Volumen (cl)', 'Tipo Fermentación',
    'Maltas', 'IBU', 'Color', 'Envase',	'OLD-REF', 'Desc',	'IBU Exacto', 'Lúpulos']]

df

,Nombre,Precio,img_url,Estilo,Origen,% Alc.,% Alc. Exacto,Otros ingredientes,Volumen (cl),Tipo Fermentación,Maltas,IBU,Color,Envase,OLD-REF,Desc,IBU Exacto,Lúpulos
0,"Boon Oude Kriek 37,5cl",7.15,https://www.labirratorium.com/19351-large_defa...,KRIEK,Bélgica,ALTO (6-9%),6.5,Cerezas Naturales,37.5 Cl,Lambic (Fermentación espontánea o salvaje),Cebada y Trigo,0-25 Amargor bajo,Rojiza,Botella,5412783053220,Cerveza de fermentación espontánea (Lambic) de...,NaN,NaN
1,Augustiner Lagerbier Hell,2.75,https://www.labirratorium.com/367-large_defaul...,MUNICH HELLES,Alemania,MEDIO (4-6%),5.2,NaN,50 Cl,Lager (Baja Fermentación),Cebada,0-25 Amargor bajo,Amarillo pálido,Botella,4105250022003,None,12,NaN
2,Schneider Aventinus Weizen-Eisbock,2.85,https://www.labirratorium.com/366-large_defaul...,NaN,Alemania,NaN,12,NaN,33,Lager (Baja Fermentación),NaN,0-25 Amargor bajo,NaN,NaN,4003669018269,"Cerveza de color medio oscuro, aromas alcohóli...",NaN,NaN
3,Aecht Schlenkerla Rauchbier Weizen,2.7,https://www.labirratorium.com/364-large_defaul...,RAUCHBIER,Alemania,MEDIO (4-6%),5.2,NaN,50 Cl,Ale (Alta Fermentación),Trigo y Cebada,0-25 Amargor bajo,Marrón,Botella,4037458000159,Rauchbier (cerveza ahumada) de trigo con 5.2% ...,20,NaN
4,Samuel Adams Boston Lager,2.45,https://www.labirratorium.com/343-large_defaul...,AMERICAN LAGER,EEUU,MEDIO (4-6%),4.8,NaN,33 Cl,Samuel Adams lager,Samuel Adams two-row pale malt blend y Caramel 60,25-50 Amargor medio,Ámbar,Botella,5012686070005,None,30,Hallertau Mittelfrueh y Tettnang Tettnanger No...
5,Westmalle Dubbel,2.5,https://www.labirratorium.com/102-large_defaul...,DUBBEL,Bélgica,ALTO (6-9%),7,NaN,33 Cl,Ale (Alta Fermentación),Cebada,None,Marrón,Botella,5412343152332,Cerveza Trapista belga con 7% ABV de color mar...,24,NaN
6,Duchesse De Bourgogne,2.95,https://www.labirratorium.com/12300-large_defa...,None,Bélgica,ALTO (6-9%),6.2,NaN,25 Cl,Ale (Alta Fermentación),Cebada,0-25 Amargor bajo,Ámbar,Botella,5411364151119,None,NaN,NaN
7,Tripel Karmeliet 33 cl.,2.7,https://www.labirratorium.com/81-large_default...,NaN,Bélgica,ALTO (6-9%),8.4,NaN,33 Cl,Ale (Alta Fermentación),"Cebada, Avena y Trigo",0-25 Amargor bajo,Amarillo pálido,Botella,54050082,None,16,NaN
8,Weihenstephaner Vitus,2.95,https://www.labirratorium.com/43-large_default...,TRIGO ALEMÁN (WEIZENDOPPELBOCK),Alemania,ALTO (6-9%),"7,7",NaN,50 Cl,Ale (Alta Fermentación),Trigo y Cebada,0-25 Amargor bajo,Amarillo dorado,Botella,4105120004108,None,NaN,NaN
9,Oud Beersel Oude Kriek Vieille 75 cl.,15.25,https://www.labirratorium.com/620-large_defaul...,KRIEK,Belgica,MEDIO (4-6%),6,Cerezas,75,Lambic (Fermentación espontánea o salvaje),Cebada y Trigo,0-25 Amargor bajo,Rojiza,Botella,5425018070138,None,NaN,NaN
